In [1]:
import sys
sys.path.append('../')

import meta_dataloader.TCGA

import models.mlp, models.gcn
import numpy as np
import data.gene_graphs
import collections
import sklearn.metrics
import sklearn.model_selection

%load_ext autoreload
%autoreload 2

In [3]:
tasks = meta_dataloader.TCGA.TCGAMeta(download=True, min_samples_per_class=10)

In [11]:
# for i, task in enumerate(tasks):
#     print (i, task.id, collections.Counter(task._labels))

In [12]:
task = tasks[113]
print(task.id)
print(task._samples.shape)
print(np.asarray(task._labels).shape)
print(collections.Counter(task._labels))

('PAM50Call_RNAseq', 'BRCA')
(956, 16300)
(956,)
Counter({2: 434, 3: 194, 0: 142, 4: 119, 1: 67})


In [18]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(task._samples, 
                                                                            task._labels, 
                                                                            stratify=task._labels,
                                                                            train_size=50,
                                                                            test_size=100,
                                                                            shuffle=True,
                                                                            random_state=0
                                                                           )

In [25]:
model = models.mlp.MLP(name="MLP_lay2_chan512", num_layer=2, channels=512, 
                       metric=sklearn.metrics.accuracy_score)


Early stopping metric is accuracy_score


In [26]:
model.fit(X_train, y_train)

  batch (0/40), train loss:1.9428
  batch (10/40), train loss:5.3253
  batch (20/40), train loss:4.2757
  batch (30/40), train loss:2.3530
epoch: 0, time: 0.04, valid_metric: 0.10, train_metric: 0.15
  batch (0/40), train loss:2.0514
  batch (10/40), train loss:1.7768
  batch (20/40), train loss:2.8266
  batch (30/40), train loss:2.3317
epoch: 1, time: 0.04, valid_metric: 0.50, train_metric: 0.45
  batch (0/40), train loss:1.2759
  batch (10/40), train loss:1.8102
  batch (20/40), train loss:1.5561
  batch (30/40), train loss:1.1205
epoch: 2, time: 0.04, valid_metric: 0.50, train_metric: 0.60
  batch (0/40), train loss:1.3134
  batch (10/40), train loss:1.2790
  batch (20/40), train loss:1.3623
  batch (30/40), train loss:1.6651
epoch: 3, time: 0.04, valid_metric: 0.30, train_metric: 0.65
  batch (0/40), train loss:1.1341
  batch (10/40), train loss:1.0878
  batch (20/40), train loss:1.2404
  batch (30/40), train loss:0.9274
epoch: 4, time: 0.04, valid_metric: 0.60, train_metric: 0.60


In [27]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)
print(sklearn.metrics.accuracy_score(y_test, y_pred))


0.84


In [31]:
graph = data.gene_graphs.GeneManiaGraph()
adj = graph.adj()

Torrent name: genemania.pkl, Size: 9.61MB


In [36]:
model = models.gcn.GCN(name="GCN_lay3_chan64_emb32_dropout_agg_hierarchy", 
                       dropout=True, 
                       cuda=True,
                       num_layer=4, 
                       channels=64, 
                       embedding=32, 
                       aggregation="hierarchy",
                       lr=0.0001,
                       num_epochs=5
                      )

Early stopping metric is accuracy_score


In [37]:
model.fit(X_train, y_train, adj)

/network/home/cohenjos/.local/lib/python3.6/site-packages/scipy/sparse/compressed.py:708: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self[i, j] = values


Reducing graph by a factor of 2 to 8150 nodes
Found cache for /network/tmp1/cohenjos/workspace/gene-graph-conv/.cache/hierarchicalb9a8a435080edb44dd7545b40248ef139ecffef54a4be93b3a89847e040373058150.npy
Reducing graph by a factor of 2 to 4075 nodes
Found cache for /network/tmp1/cohenjos/workspace/gene-graph-conv/.cache/hierarchicalbb9e42c8c4f9f496dd14aa64465f6321b54a97ddd7d02b7252ba9eebc32acdb14075.npy
Reducing graph by a factor of 2 to 2037 nodes
Found cache for /network/tmp1/cohenjos/workspace/gene-graph-conv/.cache/hierarchical9ce023dfad768dca97b2f9074706b0e7a41c70fa35014204c21d6665325cb2bb2037.npy
Reducing graph by a factor of 2 to 1018 nodes
Found cache for /network/tmp1/cohenjos/workspace/gene-graph-conv/.cache/hierarchicalc7e9ed6e28ab56bbbf992ab185873e96f035f1c695f8e8aba35af49c10ce58281018.npy
  batch (0/40), train loss:1.6127
  batch (10/40), train loss:1.5421
  batch (20/40), train loss:1.5493
  batch (30/40), train loss:1.5158
epoch: 0, time: 0.28, valid_metric: 0.50, train_m

In [40]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)
print(sklearn.metrics.accuracy_score(y_test, y_pred))

0.45
